In [1]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

from PIL import Image
import sys
from pathlib import Path
from PIL import Image
import torch
from molmo_utils import load_model, do_inference, get_coordinates, calculate_probability_of_coordinates
from utils import extract_all, plot_scatter, calculate_normalized_euclidian_distance, calculate_spread

2025-02-06 12:52:22.773300: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 12:52:23.169207: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-06 12:52:23.169249: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-06 12:52:23.169281: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 12:52:23.186281: I tensorflow/core/platform/cpu_feature_g

In [2]:
model_name = 'allenai/Molmo-7B-D-0924'
model, processor = load_model(model_name)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
def calculate_spread2(distances, width, height):
    max_dist = 0
    len_coords = len(distances)
    
    for i in range(len_coords):
        for j in range(len_coords):
            spread = calculate_normalized_euclidian_distance(coordinates[i], coordinates[j], width, height)
            if (spread > max_dist):
                max_dist = spread
    return max_dist

In [13]:
import numpy as np
from scipy.spatial import ConvexHull, distance
def calculate_spread2(points, width, height):
    """
    Computes the maximum Euclidean distance between two farthest points in a given list.
    :param points: List or numpy array of (x, y) coordinates.
    :return: Maximum Euclidean distance.
    """
    points = np.array(points)
    # Compute convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    # Compute pairwise distances only among hull points
    max_dist = 0
    for i in range(len(hull_points)):
        for j in range(i + 1, len(hull_points)):
            max_dist = max(max_dist, calculate_normalized_euclidian_distance(hull_points[i], hull_points[j], width, height))
    return max_dist

In [22]:
def run_experiment(path_to_experiments, prompt_starts, prompt_ends, runs_per_task, temperature):
    tasks, images, ground_truths, _ = extract_all(path_to_experiments)
    softmax_fn = torch.nn.Softmax(dim=-1)

    distances = []
    spreads = []

    distances_what = []
    distances_where = []

    spreads_what = []
    spreads_where = []
    
    for i in range(len(tasks)):
        task = tasks[i]
        ground_truth = ground_truths[i]
        prompt_what = prompt_starts[0] + task + prompt_ends[0]
        prompt_where = prompt_starts[1] + task + prompt_ends[1]
        print(f"Task: {task}")
    
        norm_dist_what = []
        norm_dist_where = []

        coord_what = []
        coord_where = []

        coordinates_of_task = []
        
        with Image.open(images[i]) as image:
            image_width = image.width
            image_height = image.height
    
            prompts = [prompt_what]
            if (len(ground_truth) == 2):
                prompts.append(prompt_where)
    
            for k in range(len(prompts)):
                for j in range(runs_per_task):
                    output_text, output, inputs = do_inference(image, prompts[k], model, processor, temperature)
                    
                    coordinates = get_coordinates(output_text, image_width, image_height)
                    if len(coordinates) == 1:
                        norm_dist = calculate_normalized_euclidian_distance(coordinates[0], ground_truth[k], image_width, image_height)

                        if (k == 0):
                            norm_dist_what.append(norm_dist)
                            coord_what.append(coordinates)
                        else:
                            norm_dist_where.append(norm_dist)
                            coord_where.append(coordinates)

    
            # Per task output
            all_distances = norm_dist_what + norm_dist_where
            mean = sum(all_distances) / len(all_distances)
            distances.append(mean)

            all_coordintaes = coord_what + coord_where
            max_dist = calculate_spread2(all_coordintaes, image_width, image_height)
            spreads.append(max_dist)
            
    
            mean_what = sum(norm_dist_what) / len(norm_dist_what)
            distances_what.append(mean_what)
            mean_where = sum(norm_dist_where) / len(norm_dist_where)
            distances_where.append(mean_where)
    
            max_dist_what = calculate_spread2(coord_what, image_width, image_height)
            spreads_what.append(max_dist_what)
    
            max_dist_where = calculate_spread2(coord_where, image_width, image_height)
            spreads_where.append(max_dist_where)
    
  

    plot_scatter(tasks, spreads, distances, f"Scatterplot for every task")
    plot_scatter(["Object to move", "Where to place"], [spreads_what, spreads_where], [distances_what, distances_where], f"Scatterplot for difference between object and where to place")

In [ ]:
experiment_path = "../data/base_experiments_fast"
num_runs = 3
temperature = 0.6

prompt_starts = ["You are the robot in the picture. Your mission it to guide and support the user in its task to: ", "You are the robot in the picture. Your mission it to guide and support the user in its task to: "]
prompt_ends = [" For that you have to point to the relevant object in the picture. Your output format should be like this: (x1, y1) with (x1, y1) beeing the coordinates of the object to move. Dont output anything else.", " For that you have to point to the place in the picture where the object has to be moved to. Your output format should be like this: (x1, y1) with (x1, y1) beeing the coordinates of the place where the object has to be moved to. Dont output anything else."]

run_experiment(experiment_path, prompt_starts, prompt_ends, num_runs, temperature)

Task: Place the pot to the right of the blue fork.
